In [12]:
import pandas as pd
import gzip

In [13]:
!pwd

/Users/diegocristobal/code/dieg9906/book_recommender/notebooks


In [14]:
data_frame=pd.read_csv("../book_recommender/data/book_id_map.csv")

In [15]:
data_frame

,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361
...,...,...
2360645,2360645,19517100
2360646,2360646,18597299
2360647,2360647,18584882
2360648,2360648,18518801


In [16]:
%%time
data_frame2=pd.read_csv("../book_recommender/data/goodreads_interactions.csv")

CPU times: user 33.2 s, sys: 9.76 s, total: 42.9 s
Wall time: 49.9 s


In [17]:
data_frame2

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0
...,...,...,...,...,...
228648337,876144,24772,0,0,0
228648338,876144,23847,1,4,0
228648339,876144,23950,1,3,0
228648340,876144,374106,1,5,1


In [18]:
liked_books = ["1885731", "330786", "330785"]

In [19]:
csv_book_mapping={}
with open("../book_recommender/data/book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id


In [20]:
len(csv_book_mapping)

2360651

In [21]:
overlap_users=set()

In [22]:
%%time
with open("../book_recommender/data/goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)

CPU times: user 3min 39s, sys: 2.01 s, total: 3min 41s
Wall time: 3min 41s


In [23]:
len(overlap_users)

513

In [8]:
%%time
recommendations_lines=[]
with open("../book_recommender/data/goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            recommendations_lines.append([user_id, book_id, rating])


CPU times: user 1min 46s, sys: 1.84 s, total: 1min 47s
Wall time: 1min 48s


In [9]:
recs = pd.DataFrame(recommendations_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [10]:
#Which book_id's occurs the most? --> that will be our recommendation
top_recommendations=recs["book_id"].head(10)

In [11]:
#Use the book_id to get the book information
top_recommendations = top_recommendations.index.values

In [7]:
books_titles = pd.read_json("../book_recommender/data/books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)




In [13]:
books_titles[books_titles["book_id"].isin(top_recommendations)]

,Unnamed: 0,book_id,title,ratings,url,cover_img,isbn,authors,description,mod_title


In [14]:
all_recs = recs["book_id"].value_counts()

In [15]:
all_recs = all_recs.to_frame().reset_index()
all_recs.columns = ["book_id", "book_count"]

In [16]:
all_recs.head(5)

,book_id,book_count
0,2767052,297
1,330785,285
2,7260188,271
3,6148028,270
4,41865,251


In [17]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [18]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])

In [19]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,Unnamed: 0,title,ratings,url,cover_img,isbn,authors,description,mod_title,score
1,330785,285,4860,Harry Potter and the Half-Blood Prince (Harry ...,1890,https://www.goodreads.com/book/show/330785.Har...,https://s.gr-assets.com/assets/nophoto/book/11...,155192756X,"[{'author_id': '1077326', 'role': ''}]","It is the middle of the summer, but there is a...",harry potter and the halfblood prince harry po...,42.976190
9,330786,187,4859,Harry Potter and the Prisoner of Azkaban (Harr...,1232,https://www.goodreads.com/book/show/330786.Har...,https://images.gr-assets.com/books/1173807336m...,1551922460,"[{'author_id': '1077326', 'role': ''}]",Harry Potter is a wizard. Along with Ron and H...,harry potter and the prisoner of azkaban harry...,28.383929
57,1885731,132,381178,Harry Potter and the Philosopher's Stone (Harr...,770,https://www.goodreads.com/book/show/1885731.Ha...,https://images.gr-assets.com/books/1387739847m...,0747549559,"[{'author_id': '1077326', 'role': ''}]",Harry Potter thinks he is an ordinary boy - un...,harry potter and the philosophers stone harry ...,22.628571
34,532846,154,240772,Harry Potter and the Goblet of Fire (Harry Pot...,1315,https://www.goodreads.com/book/show/532846.Har...,https://s.gr-assets.com/assets/nophoto/book/11...,1551923378,"[{'author_id': '1077326', 'role': ''}]","It is the summer holidays, and one night Harry...",harry potter and the goblet of fire harry pott...,18.034981
35,532837,153,132394,Harry Potter and the Order of the Phoenix (Har...,1507,https://www.goodreads.com/book/show/532837.Har...,https://images.gr-assets.com/books/1463561210m...,1551925702,"[{'author_id': '1077326', 'role': ''}]",Dumbledore lowered his hands and surveyed Harr...,harry potter and the order of the phoenix harr...,15.533510
85,480478,111,111750,Harry Potter and the Chamber of Secrets (Harry...,2414,https://www.goodreads.com/book/show/480478.Har...,https://images.gr-assets.com/books/1328105970m...,1551922444,"[{'author_id': '1077326', 'role': ''}]","Harry, Ron and Hermione have returned to Hogwa...",harry potter and the chamber of secrets harry ...,5.103977
297,1045206,64,308724,Harry Potter and the Chamber of Secrets (Harry...,894,https://www.goodreads.com/book/show/1045206.Ha...,https://images.gr-assets.com/books/1327108956m...,155192370X,"[{'author_id': '1077326', 'role': ''}]",The Dursleys were so mean and hideous that sum...,harry potter and the chamber of secrets harry ...,4.581655
6,818056,219,58555,Harry Potter and the Deathly Hallows (Harry Po...,13938,https://www.goodreads.com/book/show/818056.Har...,https://images.gr-assets.com/books/1351958236m...,1551929767,"[{'author_id': '1077326', 'role': ''}]",Harry Potter is leaving Privet Drive for the l...,harry potter and the deathly hallows harry pot...,3.441025
490,331562,49,114769,Harry Potter and the Chamber of Secrets (Harry...,750,https://www.goodreads.com/book/show/331562.Har...,https://images.gr-assets.com/books/1397480745m...,0747549605,"[{'author_id': '1077326', 'role': ''}]",Harry Potter is a wizard. He is in the second ...,harry potter and the chamber of secrets harry ...,3.201333
337,858352,59,243858,Harry Potter and the Philosopher's Stone (Harr...,1218,https://www.goodreads.com/book/show/858352.Har...,https://images.gr-assets.com/books/1327537793m...,155192398X,"[{'author_id': '1077326', 'role': ''}]",Harry Potter is living a miserable existence w...,harry potter and the philosophers stone harry ...,2.857964


In [20]:
all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False).head(10)

,book_id,book_count,Unnamed: 0,title,ratings,url,cover_img,isbn,authors,description,mod_title,score
1,330785,285,4860,Harry Potter and the Half-Blood Prince (Harry ...,1890,https://www.goodreads.com/book/show/330785.Har...,https://s.gr-assets.com/assets/nophoto/book/11...,155192756X,"[{'author_id': '1077326', 'role': ''}]","It is the middle of the summer, but there is a...",harry potter and the halfblood prince harry po...,42.976190
6,818056,219,58555,Harry Potter and the Deathly Hallows (Harry Po...,13938,https://www.goodreads.com/book/show/818056.Har...,https://images.gr-assets.com/books/1351958236m...,1551929767,"[{'author_id': '1077326', 'role': ''}]",Harry Potter is leaving Privet Drive for the l...,harry potter and the deathly hallows harry pot...,3.441025
2,7260188,271,22042,"Mockingjay (The Hunger Games, #3)",1743362,https://www.goodreads.com/book/show/7260188-mo...,https://images.gr-assets.com/books/1358275419m...,0439023513,"[{'author_id': '153394', 'role': ''}]",My name is Katniss Everdeen.\nWhy am I not dea...,mockingjay the hunger games 3,0.042126
3,6148028,270,243248,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,0439023491,"[{'author_id': '153394', 'role': ''}]",Sparks are igniting.\nFlames are spreading.\nA...,catching fire the hunger games 2,0.039305
5,11870085,249,320191,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,0525478817,"[{'author_id': '1406384', 'role': ''}]","There is an alternate cover edition .\n""I fel...",the fault in our stars,0.025522
7,5470,208,178658,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,0451524934,"[{'author_id': '3706', 'role': ''}, {'author_i...","The year 1984 has come and gone, but George Or...",1984,0.021376
0,2767052,297,84182,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,0439023483,"[{'author_id': '153394', 'role': ''}]",Winning will make you famous.\nLosing means ce...,the hunger games the hunger games 1,0.018002
4,41865,251,95313,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,0316015849,"[{'author_id': '941441', 'role': ''}]",About three things I was absolutely positive.\...,twilight twilight 1,0.015984


In [21]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [22]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,Unnamed: 0,title,ratings,url,cover_img,isbn,authors,description,mod_title,score
6,818056,219,58555,"Harry Potter and the Deathly Hallows (Harry Potter, #7)",13938,Goodreads,https://images.gr-assets.com/books/1351958236m/818056.jpg,1551929767,"[{'author_id': '1077326', 'role': ''}]","Harry Potter is leaving Privet Drive for the last time. But as he climbs into the sidecar of Hagrid's motorbike and they take to the skies, he knows Lord Voldemort and the Death Eaters will not be far behind. The protective charm that has kept him safe until now is broken. But the Dark Lord is breathing fear into everything he loves. And he knows he can't keep hiding. To stop Voldemort, Harry knows he must find the remaining Horcruxes and destroy them. He will have to face his enemy in one final battle. --jkrowling.com",harry potter and the deathly hallows harry potter 7,3.441025
2,7260188,271,22042,"Mockingjay (The Hunger Games, #3)",1743362,Goodreads,https://images.gr-assets.com/books/1358275419m/7260188.jpg,0439023513,"[{'author_id': '153394', 'role': ''}]","My name is Katniss Everdeen. Why am I not dead? I should be dead. Katniss Everdeen, girl on fire, has survived, even though her home has been destroyed. Gale has escaped. Katniss's family is safe. Peeta has been captured by the Capitol. District 13 really does exist. There are rebels. There are new leaders. A revolution is unfolding. It is by design that Katniss was rescued from the arena in the cruel and haunting Quarter Quell, and it is by design that she has long been part of the revolution without knowing it. District 13 has come out of the shadows and is plotting to overthrow the Capitol. Everyone, it seems, has had a hand in the carefully laid plans--except Katniss. The success of the rebellion hinges on Katniss's willingness to be a pawn, to accept responsibility for countless lives, and to change the course of the future of Panem. To do this, she must put aside her feelings of anger and distrust. She must become the rebels' Mockingjay--no matter what the personal cost.",mockingjay the hunger games 3,0.042126
3,6148028,270,243248,"Catching Fire (The Hunger Games, #2)",1854746,Goodreads,https://images.gr-assets.com/books/1358273780m/6148028.jpg,0439023491,"[{'author_id': '153394', 'role': ''}]","Sparks are igniting. Flames are spreading. And the Capitol wants revenge. Against all odds, Katniss has won the Hunger Games. She and fellow District 12 tribute Peeta Mellark are miraculously still alive. Katniss should be relieved, happy even. After all, she has returned to her family and her longtime friend, Gale. Yet nothing is the way Katniss wishes it to be. Gale holds her at an icy distance. Peeta has turned his back on her completely. And there are whispers of a rebellion against the Capitol - a rebellion that Katniss and Peeta may have helped create. Much to her shock, Katniss has fueled an unrest she's afraid she cannot stop. And what scares her even more is that she's not entirely convinced she should try. As time draws near for Katniss and Peeta to visit the districts on the Capitol's cruel Victory Tour, the stakes are higher than ever. If they can't prove, without a shadow of a doubt, that they are lost in their love for each other, the consequences will be horrifying. In Catching Fire, the second novel in the Hunger Games trilogy, Suzanne Collins continues the story of Katniss Everdeen, testing her more than ever before...and surprising readers at every turn.",catching fire the hunger games 2,0.039305
5,11870085,249,320191,The Fault in Our Stars,2429317,Goodreads,https://images.gr-assets.com/books/1360206420m/11870085.jpg,0525478817,"[{'author_id': '1406384', 'role': ''}]","There is an alternate cover edition . ""I fell in love the way you fall asleep: slowly, then all at once."" Despite the tumor-shrinking medical miracle that has bought her a few years, Hazel has never been anything but terminal, her final chapter inscribed upon diagnosis. But when a gorgeous